<a href="https://colab.research.google.com/github/Thomas-J-Kidd/ECEN4493_transfer_learning_examples/blob/main/transfer_learning_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import Necessary Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt


# Step 2: Define Data Preprocessing
- Data preprocessing steps are defined to resize images for the model, convert them to tensors, and normalize their colors.

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit MobileNetV2 input dimensions
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize images
])


# Step 3: Load Dataset
-     CIFAR-10 dataset is loaded with transformations applied.
- Data loaders are created for both training and testing datasets with a specified batch size.

In [3]:
# Assuming you're using CIFAR-10, but PyTorch's dataset class can be used for many other datasets.
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 170498071/170498071 [00:13<00:00, 13089889.22it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Step 4: Load Pre-trained Model and Modify It
-     Load a pre-trained MobileNetV2 model.
- Freeze its parameters to prevent them from being updated during training.
- Modify the classifier layer to match the number of classes in the CIFAR-10 dataset.

In [4]:
model = models.mobilenet_v2(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

# Modify the last layer
model.classifier[1] = nn.Linear(model.last_channel, 10)  # Assuming 10 classes


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 127MB/s]


# Step 5: Define Loss Function and Optimizer
-     CrossEntropyLoss is used for multi-class classification.
- The Adam optimizer is set to optimize only the parameters of the last layer, which speeds up training and saves resources.


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[1].parameters(), lr=0.001)  # Optimize only the last layer


# Step 6: Train the Model
-     Train the model for a defined number of epochs.
- Iterate over the training dataset, compute the loss for each batch, and update the model's weights.

In [7]:
num_epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
inputs, labels = inputs.to(device), labels.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")



Epoch 1, Loss: 0.7355582118034363
Epoch 2, Loss: 0.6261181831359863
Epoch 3, Loss: 0.6743506193161011
Epoch 4, Loss: 0.5799391269683838
Epoch 5, Loss: 0.7747159004211426


# Step 7: Evaluate the Model

-     Evaluate the model's performance on the test dataset.
- Compute the overall accuracy by comparing the model's predictions to the true labels.

In [11]:
model = model.to(device)
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total}%')


Accuracy of the model on the test images: 78.02%
